In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_23372\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_23372\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [11]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [12]:
df.cat.tail(20)

,Soil Type,Crop Type,Fertilizer Name
749980,0,10,5
749981,2,2,2
749982,3,8,0
749983,4,3,2
749984,4,6,1
749985,3,3,0
749986,0,10,3
749987,3,3,6
749988,0,3,1
749989,0,4,4


In [13]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [14]:
matrix=df_train.corr()

In [15]:
matrix

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type,Crop Type,Fertilizer Name
id,1.000000,-0.001069,-0.000310,0.001752,0.001505,-0.000375,-0.001234,0.002831,-0.000953,-0.000179
Temparature,-0.001069,1.000000,0.001758,-0.001633,-0.006073,-0.009913,0.000298,-0.007621,-0.006753,0.001727
Humidity,-0.000310,0.001758,1.000000,0.003424,0.006502,0.005229,0.002413,0.003921,0.002414,0.010626
Moisture,0.001752,-0.001633,0.003424,1.000000,0.003016,-0.001931,-0.006735,-0.003941,-0.003400,0.011493
Nitrogen,0.001505,-0.006073,0.006502,0.003016,1.000000,0.000543,-0.005464,-0.000650,0.001611,-0.004725
Potassium,-0.000375,-0.009913,0.005229,-0.001931,0.000543,1.000000,0.000113,0.005314,-0.003748,-0.010872
Phosphorous,-0.001234,0.000298,0.002413,-0.006735,-0.005464,0.000113,1.000000,-0.000275,0.000694,-0.006127
Soil Type,0.002831,-0.007621,0.003921,-0.003941,-0.000650,0.005314,-0.000275,1.000000,-0.006213,0.016576
Crop Type,-0.000953,-0.006753,0.002414,-0.003400,0.001611,-0.003748,0.000694,-0.006213,1.000000,-0.013041
Fertilizer Name,-0.000179,0.001727,0.010626,0.011493,-0.004725,-0.010872,-0.006127,0.016576,-0.013041,1.000000


In [16]:
X = df_train[["Temparature","Humidity","Moisture","Crop Type","Soil Type"]]

In [17]:
y = df_train[["Fertilizer Name"]]

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import Dataset

class FeatureTokenizer(nn.Module):
    def __init__(self, num_numeric, cat_cardinalities, embed_dim):
        super().__init__()
        self.num_linear = nn.Linear(num_numeric, embed_dim)
        self.cat_embeds = nn.ModuleList([
            nn.Embedding(cardinality, embed_dim) for cardinality in cat_cardinalities
        ])
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))

    def forward(self, x_num, x_cat):
        # Sanity check
        assert x_cat.shape[1] == len(self.cat_embeds), (
            f"x_cat has {x_cat.shape[1]} columns, but cat_embeds has {len(self.cat_embeds)} embeddings"
        )

        # Project numerical features
        num_tokens = self.num_linear(x_num).unsqueeze(1)  # [B, 1, D]

        # Clamp categorical indices to valid range
        x_cat_clamped = torch.stack([
            torch.clamp(x_cat[:, i], max=self.cat_embeds[i].num_embeddings - 1)
            for i in range(x_cat.shape[1])
        ], dim=1)

        # Embed categorical features
        cat_tokens = [embed(x_cat_clamped[:, i])[:, None, :] for i, embed in enumerate(self.cat_embeds)]

        # Concatenate [CLS] + categorical + numerical tokens
        tokens = torch.cat(
            [self.cls_token.expand(x_num.size(0), -1, -1)] + cat_tokens + [num_tokens],
            dim=1
        )  # [B, T, D]

        return tokens

In [20]:
import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import Dataset
class FTTransformer(nn.Module):
    def __init__(self, num_numeric, cat_cardinalities, embed_dim=32, num_heads=4, num_layers=2, num_classes=7):
        super().__init__()
        cat_cardinalities = [
    df["Soil Type"].nunique(),
    df["Crop Type"].nunique()]
        self.tokenizer = FeatureTokenizer(num_numeric=3, cat_cardinalities=cat_cardinalities, embed_dim=embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x_num, x_cat):
        tokens = self.tokenizer(x_num, x_cat)          # includes [CLS] token already
        encoded = self.transformer(tokens)
        return self.head(encoded[:, 0, :])              # pull [CLS] token output

In [21]:
class FTTransformer(nn.Module):
    def __init__(self, num_numeric, cat_cardinalities, embed_dim=64, num_heads=4, num_layers=4, num_classes=7):
        super().__init__()
        self.tokenizer = FeatureTokenizer(num_numeric, cat_cardinalities, embed_dim)
#PyTorch uses the Post-LN Transformer architecture:
#It adds residual connections around both the multi-head attention and feedforward blocks.
#It applies layer normalization after each residual connection.

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True,
            activation='gelu'  # smoother gradients than ReLU
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x_num, x_cat):
        tokens = self.tokenizer(x_num, x_cat)  # [B, T, D] with [CLS] already included
        encoded = self.transformer(tokens)
        return self.head(encoded[:, 0, :])     # pull from [CLS] token

In [22]:
numeric_features = ["Temperature", "Humidity", "Moisture"]  # update as needed
num_numeric = len(numeric_features)

In [23]:
cat_cardinalities = [
    df["Soil Type"].nunique(),
    df["Crop Type"].nunique()
]
# Encode string categories if not already done
df["Soil Type"] = df["Soil Type"].astype("category").cat.codes
df["Crop Type"] = df["Crop Type"].astype("category").cat.codes
df["Fertilizer Name"] = df["Fertilizer Name"].astype("category").cat.codes
# Create the categorical tensor
x_cat_tensor = torch.tensor(df[["Soil Type", "Crop Type"]].values, dtype=torch.long)
y_tensor = torch.tensor(df[["Fertilizer Name"]].values, dtype=torch.long)

In [24]:
model = FTTransformer(
    num_numeric=num_numeric,
    cat_cardinalities=cat_cardinalities,
    embed_dim=64,
    num_heads=4,
    num_layers=4,
    num_classes=7
).to(device)

In [25]:
# Features
x_cat_tensor = torch.tensor(df[["Soil Type", "Crop Type"]].values, dtype=torch.long)
x_num_tensor = torch.tensor(df[["Temparature", "Humidity", "Moisture"]].values, dtype=torch.float32)
# Target
y_train = torch.tensor(df["Fertilizer Name"].values, dtype=torch.long)

In [26]:
y_tensor = y_train.squeeze()  # shape: [750000]

In [27]:
#fertilizer_classes = pd.Categorical(df["Fertilizer Name"]).categories.tolist()

In [28]:

print("x_num:", x_num_tensor.shape)
print("x_cat:", x_cat_tensor.shape)
print("y_cat:", y_tensor.shape)

x_num: torch.Size([750000, 3])
x_cat: torch.Size([750000, 2])
y_cat: torch.Size([750000])


In [29]:
criterion = torch.nn.CrossEntropyLoss()

In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

C:\Users\pkitt\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [31]:
print(type(x_num_tensor), type(x_cat_tensor), type(y_tensor))

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [32]:

class FertilizerDataset(Dataset):
    def __init__(self, x_num, x_cat, y):
        self.x_num = x_num
        self.x_cat = x_cat
        self.y = y  # y must be a LongTensor

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # ✅ Return three tensors — no tuple wrapping!
        return self.x_num[idx], self.x_cat[idx], self.y[idx]

In [33]:
from torch.utils.data import DataLoader

dataset = FertilizerDataset(x_num_tensor, x_cat_tensor, y_tensor)  # make sure y_train is aligned
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [34]:
#1. Prep: Activation Stats Collecto
from collections import defaultdict

activation_stats = defaultdict(list)

def record_activation(name):
    def hook(module, input, output):
        activation_stats[name].append(output.detach().abs().max(dim=0).values)
    return hook

In [35]:
#Auto-Hook Target Layer
def hook_transformer_layers(model):
    for i, layer in enumerate(model.transformer.layers):
        layer.linear1.register_forward_hook(record_activation(f"layer_{i}_linear1"))
        layer.linear2.register_forward_hook(record_activation(f"layer_{i}_linear2"))
        layer.self_attn.out_proj.register_forward_hook(record_activation(f"layer_{i}_attn_out"))

In [36]:
# collect stacks from callibration data
def run_calibration(model, dataloader, num_batches=4):
    model.eval()
    with torch.no_grad():
        for i, (x, _) in enumerate(dataloader):
            if i >= num_batches: break
            _ = model(x)

In [37]:
# computing scaling factors
def compute_scaling_factors(alpha=0.5, eps=1e-6):
    scaling_factors = {}
    for name, acts in activation_stats.items():
        max_act = torch.stack(acts).max(dim=0).values
        scaling_factors[name] = max_act.pow(alpha) + eps
    return scaling_factors

In [38]:
# smoothquant Wrapper
class SmoothQuantWrapper(torch.nn.Module):
    def __init__(self, linear, scale):
        super().__init__()
        self.linear = linear
        self.register_buffer("scale", scale)

    def forward(self, x):
        x = x / self.scale
        W = self.linear.weight * self.scale.unsqueeze(0)
        return torch.nn.functional.linear(x, W, self.linear.bias)

In [39]:
# wrap target layer
def apply_smoothquant(model, scaling_factors):
    for i, layer in enumerate(model.transformer.layers):
        for tag in ["linear1", "linear2", "attn_out"]:
            name = f"layer_{i}_{tag}"
            if name in scaling_factors:
                scale = scaling_factors[name]
                linear_layer = getattr(layer, tag) if tag != "attn_out" else layer.self_attn.out_proj
                wrapped = SmoothQuantWrapper(linear_layer, scale)
                if tag == "attn_out":
                    layer.self_attn.out_proj = wrapped
                else:
                    setattr(layer, tag, wrapped)

In [40]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, Subset
import random

# Load and preprocess
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")
# Fix column name typo and ensure list format
feature_cols = ["Temparature", "Humidity", "Moisture", "Crop Type", "Soil Type"]
X_df = df[feature_cols]
X_encoded = pd.get_dummies(X_df)  # one-hot encode categorical inputs

from sklearn.preprocessing import LabelEncoder

# Label encode the fertilizer names (target)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df["Fertilizer Name"])  # ← now this is numeric

X_encoded = X_encoded.astype("float32")  # Ensures correct dtype
# Convert to tensors
X_tensor = torch.tensor(X_encoded.values, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

# Create the dataset and loader
full_dataset = TensorDataset(X_tensor, y_tensor)
calibration_indices = random.sample(range(len(full_dataset)), 512)
calibration_dataset = Subset(full_dataset, calibration_indices)
calibration_loader = DataLoader(calibration_dataset, batch_size=64, shuffle=False)


In [41]:
from collections import defaultdict

activation_stats = defaultdict(list)

def record_activation(name):
    def hook(module, input, output):
        activation_stats[name].append(output.detach().abs().max(dim=0).values)
    return hook

def hook_transformer_layers(model):
    for i, layer in enumerate(model.transformer.layers):
        layer.linear1.register_forward_hook(record_activation(f"layer_{i}_linear1"))
        layer.linear2.register_forward_hook(record_activation(f"layer_{i}_linear2"))
        layer.self_attn.out_proj.register_forward_hook(record_activation(f"layer_{i}_attn_out"))

In [42]:
def run_calibration(model, dataloader, num_batches=4):
    model.eval()
    with torch.no_grad():
        for i, (inputs, _) in enumerate(dataloader):
            if i >= num_batches:
                break
            x_num, x_cat = inputs[0], inputs[1]  # <- unpacking the list
            _ = model(x_num, x_cat)

In [43]:
def compute_scaling_factors(alpha=0.5, eps=1e-6):
    scaling_factors = {}
    for name, acts in activation_stats.items():
        max_act = torch.stack(acts).max(dim=0).values
        scaling_factors[name] = max_act.pow(alpha) + eps
    return scaling_factors

In [44]:
class SmoothQuantWrapper(torch.nn.Module):
    def __init__(self, linear, scale):
        super().__init__()
        self.linear = linear
        self.register_buffer("scale", scale)

    def forward(self, x):
        x = x / self.scale
        W = self.linear.weight * self.scale.unsqueeze(0)
        return torch.nn.functional.linear(x, W, self.linear.bias)

def apply_smoothquant(model, scaling_factors):
    for i, layer in enumerate(model.transformer.layers):
        for tag in ["linear1", "linear2", "attn_out"]:
            name = f"layer_{i}_{tag}"
            if name in scaling_factors:
                scale = scaling_factors[name]
                target_layer = layer.self_attn.out_proj if tag == "attn_out" else getattr(layer, tag)
                wrapper = SmoothQuantWrapper(target_layer, scale)
                if tag == "attn_out":
                    layer.self_attn.out_proj = wrapper
                else:
                    setattr(layer, tag, wrapper)

In [45]:
# Separate numerical and categorical features
num_features = ["Temparature", "Humidity", "Moisture"]
cat_features = ["Crop Type", "Soil Type"]

# Encode categorical as integer indices
df_cat = df[cat_features].astype("category")
X_cat_tensor = torch.tensor(df_cat.apply(lambda col: col.cat.codes).values, dtype=torch.long)

# Numerical inputs
X_num_tensor = torch.tensor(df[num_features].values, dtype=torch.float32)

# Label encode the targets
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_tensor = torch.tensor(label_encoder.fit_transform(df["Fertilizer Name"]), dtype=torch.long)

# Wrap in a custom tuple-returning dataset
from torch.utils.data import TensorDataset, Subset, DataLoader
full_dataset = TensorDataset(X_num_tensor, X_cat_tensor, y_tensor)

In [46]:
import random
calibration_indices = random.sample(range(len(full_dataset)), 512)

# Wrap with a loader that returns (x_num, x_cat), y
class FTWrapper(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        x_num, x_cat, y = self.dataset[idx]
        return (x_num, x_cat), y

calibration_dataset = FTWrapper(Subset(full_dataset, calibration_indices))
calibration_loader = DataLoader(calibration_dataset, batch_size=64, shuffle=False)

In [47]:
# Hook layers
hook_transformer_layers(model)

# Run on calibration data
run_calibration(model, calibration_loader)

# Get scaling factors
factors = compute_scaling_factors(alpha=0.5)

# Replace with scaled modules
apply_smoothquant(model, factors)

In [48]:
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x_input, y in dataloader:
            # If x_input is already a tuple of (x_num, x_cat), unpack it
            if isinstance(x_input, (list, tuple)) and len(x_input) == 2:
                x_num, x_cat = x_input
            else:
                raise ValueError("Expected input as (x_num, x_cat), but got single tensor.")

            logits = model(x_num, x_cat)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds)
            all_labels.append(y)

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    return accuracy_score(all_labels.cpu(), all_preds.cpu())

In [49]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Split raw dataframe first (so both num + cat stay aligned)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df["Fertilizer Name"])

In [50]:
class FTFertilizerDataset(torch.utils.data.Dataset):
    def __init__(self, df, num_cols, cat_cols, label_col):
        self.x_num = torch.tensor(df[num_cols].values, dtype=torch.float32)
        df_cat = df[cat_cols].astype("category")
        self.x_cat = torch.tensor(df_cat.apply(lambda col: col.cat.codes).values, dtype=torch.long)
        self.y = torch.tensor(LabelEncoder().fit_transform(df[label_col]), dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return (self.x_num[idx], self.x_cat[idx]), self.y[idx]

In [51]:
num_cols = ["Temparature", "Humidity", "Moisture"]
cat_cols = ["Crop Type", "Soil Type"]

test_dataset = FTFertilizerDataset(df_test, num_cols, cat_cols, "Fertilizer Name")
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [53]:
from sklearn.metrics import accuracy_score
acc_post_quant = evaluate_model(model, test_loader)
print(f"🎯 Accuracy after SmoothQuant: {acc_post_quant:.4f}")

🎯 Accuracy after SmoothQuant: 0.1472
